# Learning to Navigate in Cities Without a Map

## 요약

* 지능을 가진 사람은 비구조화된(unstructured) 환경에서 장거리 길찾기(Long-range Navigate)를 지도 없이도 할 수 있다.
* 장거리 길찾기는 다음 요소를 내포한 매우 복잡한 인지 task
  * 공간에 대한 잠재 표현의 학습
  * 인식 가능한 랜드마크에 대한 인지
  * 강건한 시각 프로세싱을 통한 자기 자신 위치 확신(I am here)과 목표 인식(I am going there)
* 우리는 end-to-end 강화학습 기반으로 도시 수준에서의 장거리 길찾기 테스크 문제를 다룬다.
  * integration of policies with locale-specific knowlege
  * dual pathway architecture for specific locale-feature and transfer learning
  * Google-streep view 데이터 이용
  * 목표 지점은 현 지점에서 수 km 떨어진 정도



## 데모

In [10]:

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/2yjWDNXYh5s" frameborder="0" allowfullscreen></iframe>')


## 기존 시각 기반 navigation 연구들와 우리와의 차이점 

* 가상 환경이거나 실제 환경이라도 제한된 공간 내, 매우 깨끗한(?) 이미지 기반 
  * navigate hourse scenes (Zhu 2017) 
  * navigate in 3D games(Lample 2017)
* RL 기반이 아닌 supervised learning 기반, geolocation task와 planning이 분리
  * DeepNav (Brahmbhatt, 2017)
  * RatSLAM (Milford 2004)
* 보조 task를 활용 
  * Dosovitskiy 2016
* 우리 연구의 특징 
  * real 환경 시각 데이터, 더럽다(with pedestrians, car, buses, weather, vegetation)
  * RL 기반이라서 label data 없어도 된다. 
  * 보조 task 활용 (방향 예측)
  * 다만 dynamic 한 요소는 다루지 못한다 


## 환경

* multiple cities
* multiple landmarks

![](./images/navigate/01.png)

* goal specified by distance from 5 landmarks
![](./images/navigate/02.png)

#### StreetView data
  * geolocated 360 panoramic images (forming undirected graph)
    * ex) in N.Y, 7200 nodes and 128,600 edges
  * use full graph
  * agent never observe underlying graph, only see RGB images (60 degree limited view)
  
#### Agent interface and Courier task
  * image x : 60 degree, RGB image, 84x84 pixel
  * 5 action : fast rotate left or right, slow rotate left or right, move forward
  * goal g : in terms of proximity to a set of fixed landmarks, softmax distances

#### Courier task (delivery service task)
  * navigate to a series of random locations in a city
  * start with random location and orientation
  * goal에 100미터 접근하면 골 성취, 다음 골 제시, 반복
  * 보상(reward) : 도착하면 1.0, sparse 
  * 1000 step이면 에피소드 종료
![](./images/navigate/03.png)



## 방법

### Goal-dependent Actor-Critic Reinforcement Learning

* Markov Decision Process
  * policy phi
  * value function
![](./images/navigate/04.png)
![](./images/navigate/05.png)
![](./images/navigate/06.png)

### Architectures

* policy and value function are parameterized by NN which shares all layers except final ouputs
* with LSTM
* three different architectures
![](./images/navigate/07.png)

* auxiliary task
  * prediction of agent's heading
  
* train
  * IMPALA (google's A3C)
  * 256/512 actors
  * sequence unrolled to length 50 (in LSTM)

### Curriculum Learning

* 쉬운 것부터 배워고, 점점 난이도를 높여가며 어려운 것도 배워간다. 
* courier task는 very sparse reward에 의해 학습이 어렵다
* CL 취지를 살려서, 처음 goal은 500m 이내로 주고, 이후에는 3.5km 로 장거리 목표를 준다. 

![](./images/navigate/08.png)


### Reward-shaping

* goal 도착시(within 100m) 1, 너무 sparse 하다. 
* early reward
  * goal과의 거리가 200m 이내인 경우, 거리가 가까워지면서 조금씩 reward를 늘린다. 

![](./images/navigate/15.png)


## 결과

### baselines
  * oracle : Upper
  * heuristic : Lower, random work
  
### GoalNav better than CityNav

![](./images/navigate/09.png)

### agent step은 목표까지의 거리에 비례했다. 즉 학습이 적절히 됐다. 

![](./images/navigate/10.png)

### 다 외운 거 아니야? 일반화 성능 어때?

* 학습셋과 테스트셋의 분리 필요
  * 약 20% 면적은 따로 떼서 학습시에는 goal로 채택되지 않도록
    * 하지만 그 면적으로 navigate through는 가능하다. 
  * 이 patch 면적을 작게(500m*500m), 중간(0.5km*0.5km), 크게(1km*1km)
  * 패치가 작으면 테스트셋에서 성능 저하 약하지만, 패치가 크면 성능 저하가 크기는 크다. 

![](./images/navigate/11.png)
![](./images/navigate/12.png)


### 도시마다 학습을 다 별도로 해야 하잖아. 전이학습이 되면 좋을텐데 

* 각각의 도시마다 별도의 goal LSTM이 두고, 각각 학습하는 것이 일반적이나 sample complexity가 높다
* 신규 도시의 학습에 기존 도시들의 학습 결과를 활용하는 전이 학습이 유용하다. 

![](./images/navigate/13.png)
![](./images/navigate/14.png)




